In [27]:
import cv2 
import numpy as np

from sklearn.metrics import pairwise


In [28]:
background = None

accumulated_weight = 0.5  #acculmulated ağılık

#roi reigion of interest


roi_top = 20
roi_bottom = 300
roi_right = 300
roi_left = 600 #kare köşeleri roi



In [37]:
#average backgorund valueyu bulan function
def calc_accum_avg(frame,accumulated_weight):
    
    global background #None olarak başlıhyor
    
    if background is None: #gelen frami buraya passlıyacak
        background = frame.copy().astype('float')
        return None
    
    #None değilse değeri varsa yapılacak else
    cv2.accumulateWeighted(frame,background,accumulated_weight) #çalışan ortalama fram değerine biriktiriyor
    
    #bu ağırlığı returnlemiyoruz global arka plana etki ediyor

In [38]:
#Eli segmentleme 

#elin köşelerini bulup segmentliyeceğiz 

def segment(frame,threshold=25):
    
    diff = cv2.absdiff(background.astype('uint8'),frame) #frame ile arkaplanın farkını hesaplayan
                       
    ret,thresholded = cv2.threshold(diff,threshold,255,cv2.THRESH_BINARY) #THRESHOLD = MİN 
                       
    image,contours,hierarchyc = cv2.findContours(thresholded.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)#konturları bulma
     
    if len(contours) == 0:
            return None     #kontor uzunluğu kontrolü 
                       
    else:  
                       #en büyük kontor el olması gerektiği için öyle varsayarak else
         hand_segment = max(contours,key=cv2.contourArea)              
         
                       
    return (thresholded,hand_segment) 

In [21]:
#Convex Hull kullanarak ellerin parmaklarını sayma

In [39]:
def count_fingers(thresholded,hand_segment):
    
    conv_hull = cv2.convexHull(hand_segment)
    
    #en keskin kenarları çekme
    
    #EN ÜST
    top = tuple(conv_hull[conv_hull[:,:,1].argmin()[0]])
    #en alt
    bottom = tuple(conv_hull[conv_hull[:,:,1].argmax()[0]])
    #sol
    left = tuple(conv_hull[conv_hull[:,:,0].argmin()[0]])
    #sağ
    right= tuple(conv_hull[conv_hull[:,:,0].argmax()[0]])
    
    #orta sağ sol
    
    cX = (left[0] + right[0]) // 2
    cY =  (top[1] + bottom[1]) // 2
    
    #mesafe hesaplaması iki nokta arası(euclidean distance)
    
    distance = pairwise.euclidean_distances([cX,cY],Y=[left,right,top,bottom])[0]
    
    max_distance = distance.max()
    
    #iki nokta arası hesabının %90 derecesi ile bir çember çizilecek derece el boyutuna göre opsiyonel
    
    radius = int(0.9*max_distance)
    circumfrance = (2*np.pi*radius)
    
    #çember roi çizme
    circular_roi = np.zeros(thresholded[:2],dtype='uint8')
    
    cv2.circle(circular_roi,cX,cY,radius,255,10)
    
    circular_roi = cv2.bitwise_and(thresholded,thresholded,mask=circular_roi)#bitwise maske 
    
    image,contours,hierarchy = cv2.findContours(circular_roi.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE) #CONTOURS
    
    #parmakları sayma fonksiyonu
    
    count = 0
    
    for cnt in contours:
        
        (x,y,w,h) = cv2.boundingRect(cnt) #konturun bounding boxlarını çekme
    
        #kondisyonlar elin merkezini bildiğimizden ötürü bilek vs. kısımlarını scanlememesi için 
        #elin merkezinden uzaklaşırsa  yazılacak
        
        out_of_wrist = (cY + (cY*0.25)) > (y+h)
        
        limit_points = ((circumfrence*0.25) > cnt.shape[0])
        
        if out_of_wrist and limit_points:
            count += 1
            
    return count       

In [40]:
#Kamerayı açma

cam = cv2.VideoCapture(0)

num_frames = 0

while True:
    
    
    ret,frame = cam.read()
    
    frame_copy = frame.copy()
    
    roi = frame[roi_top:roi_bottom,roi_right:roi_left]
    
    gray = cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
    
    gray = cv2.GaussianBlur(gray,(7,7),0)
    
    if num_frames < 60:
        calc_accum_avg(gray,accumulated_weight)
        
        if num_frames <=59:
            cv2.putText(frame_copy,'WAIT GETTING BACKGROUND',(200,300),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)
            cv2.imshow('Finger Count',frame_copy)
            
    else:
        
        hand = segment(gray)
        
        if hand is not None:
            
            thresholded, hand_segment = hand
            #gerçek elin etrafına konturlar çiziyor canlı video görüntüsünde
            cv2.drawContours(frame_copy,[hand_segment+(roi_right,roi_top)],-1,(255,0,0),5)
            #parmakları sayma
            fingers = count_fingers(thresholded,hand_segment)
            #sayım sonucunu displayleme
            cv2.putText(frame_copy,str(fingers),(70,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
            
            cv2.imshow('Thresholded',thresholded)
    cv2.rectangle(frame_copy,(roi_left,roi_top),(roi_right,roi_bottom),(0,0,255),5)#dışarıda kare çizicez roi ye  
    num_frames += 1
    
    cv2.imshow('Finger Count',frame_copy)
    
    k = cv2.waitKey(1) & 0xFF
    
    if k == 27:
        break
cam.release()
cv2.destroyAllWindows()

IndexError: invalid index to scalar variable.